In [ ]:
!pip install playwright nest_asyncio
!playwright install chromium
!apt-get install libatk1.0-0 libatk-bridge2.0-0 libatspi2.0-0 libxcomposite1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 MB 9.0 MB/s eta 0:00:00
(node:9420) [DEP0169] DeprecationWarning: `url.parse()` behavior is not standardized and prone to errors that have security implications. Use the WHATWG URL API instead. CVEs are not issued for `url.parse()` vulnerabilities.
(Use `node --trace-deprecation ...` to show where the warning was created)
164.7 MiB [] 0% 0.0s164.7 MiB [] 0% 19.9s164.7 MiB [] 0% 14.5s164.7 MiB [] 0% 9.8s164.7 MiB [] 0% 8.6s164.7 MiB [] 1% 7.6s164.7 MiB [] 1% 7.3s164.7 MiB [] 1% 6.9s164.7 MiB [] 2% 6.2s164.7 MiB [] 2% 5.7s164.7 MiB [] 3% 5.3s164.7 MiB [] 3% 5.2s164.7 MiB [] 3% 5.0s164.7 MiB [] 4% 5.2s164.7 MiB [] 4% 5.5s164.7 MiB [] 5% 4.7s164.7 MiB [] 5% 4.5s164.7 MiB [] 6% 4.1s164.7 MiB [] 7% 3.9s164.7 MiB [] 7% 3.8s164.7 MiB [] 8% 3.8s164.7 MiB [] 8% 3.9s164.7 MiB [] 9% 3.9s164.7 MiB [] 9% 4.0s164.7 MiB [] 10% 4.0s164.7 MiB [] 10% 3.9s164.7 MiB [] 11% 3.8s164.7 MiB [] 11% 3.7s164.7 MiB [] 12% 3.7s164.7 MiB [] 12% 3.6s164.7 MiB [] 1

In [ ]:
import asyncio, json, csv, time
from pathlib import Path
import nest_asyncio
nest_asyncio.apply()
from playwright.async_api import async_playwright

#AJAX_URL = "https://webscraper.io/test-sites/e-commerce/ajax"  # target page
#AJAX_URL = "https://webscraper.io/test-sites"
AJAX_URL = "https://webscraper.io/test-sites/e-commerce/static/computers/laptops?page=1"
async def scrape_ajax_site():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)   # set False to watch it
        ctx = await browser.new_context()
        page = await ctx.new_page()
        await page.goto(AJAX_URL, timeout=60000)

        # Wait for initial product grid
        await page.wait_for_selector(".thumbnail", timeout=30000)

        # Keep clicking "Load more" until the button disappears or is disabled
        while True:
            try:
                # Button sometimes labeled "Load more" or has class .btn-load-more
                load_more = await page.query_selector("button:has-text('Load more')")
                if not load_more:
                    break
                is_disabled = await load_more.get_attribute("disabled")
                if is_disabled:
                    break

                # Click and wait for new items to appear
                before_count = len(await page.query_selector_all(".thumbnail"))
                await load_more.click()
                # Wait for network idle and more thumbnails to load
                await page.wait_for_load_state("networkidle")
                # Soft wait: poll until count increases or timeout
                for _ in range(30):
                    after_count = len(await page.query_selector_all(".thumbnail"))
                    if after_count > before_count:
                        break
                    await asyncio.sleep(0.2)
                # If no increase, assume end
                if after_count <= before_count:
                    break

            except Exception:
                # If the button isn't present or any error occurs, exit the loop
                break

        # Extract product cards
        cards = await page.query_selector_all(".thumbnail")
        rows = []

        for card in cards:
            # Title text and product link
            title_el = await card.query_selector(".title")
            title = (await title_el.text_content()).strip() if title_el else None
            url = await title_el.get_attribute("href") if title_el else None
            # Price
            price_el = await card.query_selector(".price")
            price = (await price_el.text_content()).strip() if price_el else None
            # Rating (count of .glyphicon-star)
            stars = await card.query_selector_all(".ratings .glyphicon-star")
            rating = len(stars) if stars else 0
            # Image
            img_el = await card.query_selector("img")
            img_src = await img_el.get_attribute("src") if img_el else None

            rows.append({
                "title": title,
                "price": price,
                "rating_stars": rating,
                "product_url": url,
                "image_url": img_src
            })

        await browser.close()
        return rows

# Run and save
data = asyncio.get_event_loop().run_until_complete(scrape_ajax_site())
print(f"Collected {len(data)} products")

# Save to CSV/JSON
Path("output").mkdir(exist_ok=True)
csv_path = Path("output/products_ajax.csv")
json_path = Path("output/products_ajax.json")

with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=data[0].keys())
    writer.writeheader()
    writer.writerows(data)

with open(json_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"Saved CSV → {csv_path}")
print(f"Saved JSON → {json_path}")

Collected 6 products
Saved CSV → output/products_ajax.csv
Saved JSON → output/products_ajax.json


In [ ]:
import asyncio, json, csv
from pathlib import Path
import nest_asyncio
nest_asyncio.apply()
from playwright.async_api import async_playwright

BASE = "https://webscraper.io"

async def scrape_entire_website():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        ctx = await browser.new_context()
        page = await ctx.new_page()

        print("🔍 Collecting all category links...")
        await page.goto(BASE + "/test-sites/e-commerce/static")

        # Get all category URLs from left menu
        category_links = await page.query_selector_all(".category-link")
        categories = [BASE + await c.get_attribute("href") for c in category_links]

        print(f"Found {len(categories)} categories")

        all_rows = []

        # Loop through each category (Laptops, Phones, Tablets…)
        for cat_url in categories:
            print(f"\n📂 Scraping category: {cat_url}")
            await page.goto(cat_url)

            # Subcategories inside each category
            sub_links = await page.query_selector_all(".subcategory-link")
            subcategories = [BASE + await s.get_attribute("href") for s in sub_links]

            if not subcategories:
                # If no subcategory, treat category as product list
                subcategories = [cat_url]

            for sub_url in subcategories:
                print(f"  📁 Subcategory: {sub_url}")

                page_no = 1
                while True:
                    url = sub_url + f"?page={page_no}"
                    await page.goto(url)
                    await page.wait_for_load_state("networkidle")

                    cards = await page.query_selector_all(".thumbnail")
                    if len(cards) == 0:
                        break

                    print(f"    ➜ Page {page_no} | {len(cards)} products")

                    for card in cards:
                        title_el = await card.query_selector(".title")
                        title = (await title_el.text_content()).strip()
                        product_url = BASE + await title_el.get_attribute("href")

                        price = (await (await card.query_selector(".price")).text_content()).strip()

                        desc_el = await card.query_selector(".description")
                        desc = (await desc_el.text_content()).strip() if desc_el else ""

                        stars = await card.query_selector_all(".ratings .glyphicon-star")
                        rating = len(stars)

                        img_el = await card.query_selector("img")
                        img_src = BASE + await img_el.get_attribute("src")

                        all_rows.append({
                            "title": title,
                            "price": price,
                            "rating": rating,
                            "description": desc,
                            "product_url": product_url,
                            "image_url": img_src,
                            "category": cat_url,
                            "subcategory": sub_url,
                            "page": page_no
                        })

                    page_no += 1

        await browser.close()
        return all_rows


# Run scraper
data = asyncio.get_event_loop().run_until_complete(scrape_entire_website())
print(f"\n✅ TOTAL PRODUCTS SCRAPED: {len(data)}")

# Save output
Path("output").mkdir(exist_ok=True)
csv_path = Path("output/all_products.csv")
json_path = Path("output/all_products.json")

with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=data[0].keys())
    writer.writeheader()
    writer.writerows(data)

with open(json_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"\n📁 Saved CSV → {csv_path}")
print(f"📁 Saved JSON → {json_path}")


🔍 Collecting all category links...
Found 2 categories

📂 Scraping category: https://webscraper.io/test-sites/e-commerce/static/computers
  📁 Subcategory: https://webscraper.io/test-sites/e-commerce/static/computers/laptops
    ➜ Page 1 | 6 products
    ➜ Page 2 | 6 products
    ➜ Page 3 | 6 products
    ➜ Page 4 | 6 products
    ➜ Page 5 | 6 products
    ➜ Page 6 | 6 products
    ➜ Page 7 | 6 products
    ➜ Page 8 | 6 products
    ➜ Page 9 | 6 products
    ➜ Page 10 | 6 products
    ➜ Page 11 | 6 products
    ➜ Page 12 | 6 products
    ➜ Page 13 | 6 products
    ➜ Page 14 | 6 products
    ➜ Page 15 | 6 products
    ➜ Page 16 | 6 products
    ➜ Page 17 | 6 products
    ➜ Page 18 | 6 products
    ➜ Page 19 | 6 products
    ➜ Page 20 | 3 products
  📁 Subcategory: https://webscraper.io/test-sites/e-commerce/static/computers/tablets
    ➜ Page 1 | 6 products
    ➜ Page 2 | 6 products
    ➜ Page 3 | 6 products
    ➜ Page 4 | 3 products

📂 Scraping category: https://webscraper.io/test-sites/e

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tabulate import tabulate

BASE_URL = "https://books.toscrape.com/"

# Rating mapping
RATING_MAP = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5
}

def get_soup(url):
    resp = requests.get(url)
    resp.encoding = "utf-8"
    return BeautifulSoup(resp.text, "html.parser")

def clean_price(price_text):
    cleaned = "".join(c for c in price_text if c.isdigit() or c == ".")
    return float(cleaned)

def get_categories():
    soup = get_soup(BASE_URL)
    cat_list = soup.find("ul", class_="nav-list").find("ul")

    categories = {
        "Books": BASE_URL + "catalogue/page-1.html"
    }

    for li in cat_list.find_all("li"):
        name = li.text.strip()
        link = BASE_URL + li.find("a")["href"]
        categories[name] = link

    return categories

def scrape_category(category_name, category_url):
    books = []

    while True:
        soup = get_soup(category_url)

        for article in soup.find_all("article", class_="product_pod"):

            title = article.h3.a["title"]
            price = clean_price(article.find("p", class_="price_color").text)
            availability = article.find("p", class_="instock availability").text.strip()

            rating_word = article.find("p")["class"][1]
            rating = RATING_MAP.get(rating_word, 0)

            product_href = article.h3.a["href"].replace("../", "")
            product_url = BASE_URL + "catalogue/" + product_href
            product_soup = get_soup(product_url)

            description = "No description"
            desc = product_soup.find("div", id="product_description")
            if desc:
                description = desc.find_next("p").text.strip()

            # Adjusted price (+10% if in stock)
            adjusted_price = round(price * 1.10, 2) if "In stock" in availability else price

            books.append({
                "Category": category_name,
                "Title": title,
                "Price": price,
                "Availability": availability,
                "Rating": rating,
                "Adjusted Price": adjusted_price,
                "Description": description
            })

        next_btn = soup.find("li", class_="next")
        if next_btn:
            next_page = next_btn.a["href"]

            if "catalogue" not in category_url:
                category_url = BASE_URL + "catalogue/" + next_page
            else:
                base = category_url.rsplit("/", 1)[0]
                category_url = f"{base}/{next_page}"

            time.sleep(0.4)
        else:
            break

    return books

def scrape_all_books():
    categories = get_categories()
    all_books = []

    for cat_name, cat_url in categories.items():
        print(f"Scraping category: {cat_name} ...")
        books = scrape_category(cat_name, cat_url)
        all_books.extend(books)

    df = pd.DataFrame(all_books)

    # ------------------------------------------------
    # Apply Discount Rules
    # ------------------------------------------------

    # 50% discount for users who did not buy any book
    df["Discounted Price (No Purchase)"] = round(df["Price"] * 0.50, 2)

    # First book discount: 20%
    df["First Book Discount Price"] = round(df["Price"] * 0.80, 2)

    # If all books are in stock → remove discounts
    if all("In stock" in a for a in df["Availability"]):
        print("\n⚠ All books are in stock → removing discounts")
        df["Discounted Price (No Purchase)"] = df["Price"]
        df["First Book Discount Price"] = df["Price"]

    # ------------------------------------------------
    # TABLE OUTPUT
    # ------------------------------------------------
    print("\n📘 SAMPLE DATA (Table with Borders):")
    table_preview = df.head(10)[["Category", "Title", "Price", "Adjusted Price", "Rating"]]
    print(tabulate(table_preview, headers="keys", tablefmt="grid"))

    # ------------------------------------------------
    # Highest Rating Book
    # ------------------------------------------------
    highest_rating = df["Rating"].max()
    best_book = df[df["Rating"] == highest_rating].iloc[0]

    print("\n🏆 HIGHEST RATED BOOK:")
    print(tabulate(
        [[best_book["Title"], best_book["Category"], best_book["Rating"], best_book["Price"]]],
        headers=["Title", "Category", "Rating", "Price"],
        tablefmt="grid"
    ))

    # ------------------------------------------------
    # Lowest Rating Book
    # ------------------------------------------------
    lowest_rating = df["Rating"].min()
    worst_book = df[df["Rating"] == lowest_rating].iloc[0]

    print("\n🔻 LOWEST RATED BOOK:")
    print(tabulate(
        [[worst_book["Title"], worst_book["Category"], worst_book["Rating"], worst_book["Price"]]],
        headers=["Title", "Category", "Rating", "Price"],
        tablefmt="grid"
    ))

    return df


# Run scraper
df_books = scrape_all_books()

df_books.to_csv("all_books_final_with_all_features.csv", index=False)
print("\nSaved: all_books_final_with_all_features.csv ✔")


Scraping category: Books ...
Scraping category: Travel ...
Scraping category: Mystery ...
Scraping category: Historical Fiction ...
Scraping category: Sequential Art ...
Scraping category: Classics ...
Scraping category: Philosophy ...
Scraping category: Romance ...
Scraping category: Womens Fiction ...
Scraping category: Fiction ...
Scraping category: Childrens ...
Scraping category: Religion ...
Scraping category: Nonfiction ...
Scraping category: Music ...
Scraping category: Default ...
Scraping category: Science Fiction ...
Scraping category: Sports and Games ...
Scraping category: Add a comment ...
Scraping category: Fantasy ...
Scraping category: New Adult ...
Scraping category: Young Adult ...
Scraping category: Science ...
Scraping category: Poetry ...
Scraping category: Paranormal ...
Scraping category: Art ...
Scraping category: Psychology ...
Scraping category: Autobiography ...
Scraping category: Parenting ...
Scraping category: Adult Fiction ...
Scraping category: Humor ..

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# -----------------------------
# NLTK SETUP
# -----------------------------
nltk.download("stopwords")
from nltk.corpus import stopwords
STOP_WORDS = stopwords.words("english")

HEADERS = {"User-Agent": "Mozilla/5.0"}
BOOKS_URL = "https://books.toscrape.com/"
BBC_NEWS_URL = "https://www.bbc.com/news"

RATING_MAP = {
    "One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5
}

# =====================================================
# STEP 1: SCRAPE BOOKS
# =====================================================
def scrape_all_books(max_books=1000):
    books = []
    page_url = BOOKS_URL

    while page_url and len(books) < max_books:
        soup = BeautifulSoup(
            requests.get(page_url, headers=HEADERS).text,
            "html.parser"
        )

        for article in soup.select("article.product_pod"):
            if len(books) >= max_books:
                break

            title = article.h3.a["title"]

            price = float(
                re.findall(r"\d+\.\d+", article.select_one(".price_color").text)[0]
            )

            rating_class = article.select_one("p.star-rating")["class"][1]
            rating = RATING_MAP.get(rating_class, 0)

            books.append({
                "title": title,
                "price": price,
                "rating": rating
            })

        next_btn = soup.select_one("li.next a")
        page_url = BOOKS_URL + next_btn["href"] if next_btn else None

    return books


# =====================================================
# STEP 2: SCRAPE BBC NEWS
# =====================================================
def scrape_bbc_news(max_news=30):
    soup = BeautifulSoup(
        requests.get(BBC_NEWS_URL, headers=HEADERS).text,
        "html.parser"
    )

    urls = []
    for a in soup.select("a[href^='/news']"):
        url = "https://www.bbc.com" + a["href"]
        if url not in urls:
            urls.append(url)
        if len(urls) >= max_news:
            break

    news_data = []
    for url in urls:
        try:
            page = BeautifulSoup(
                requests.get(url, headers=HEADERS, timeout=5).text,
                "html.parser"
            )

            paragraphs = [p.text.strip() for p in page.select("article p")]
            description = " ".join(paragraphs)

            if len(description) < 50:
                continue

            news_data.append({
                "url": url,
                "description": description
            })
        except:
            continue

    return news_data


# =====================================================
# STEP 3 & 4: COSINE SIMILARITY + DISCOUNT
# =====================================================
def apply_discount_using_cosine(books, news, threshold=0.15):

    # Combine all news text
    news_text = " ".join(n["description"] for n in news)
    news_text = re.sub(r"[^a-zA-Z ]", "", news_text).lower()

    documents = [news_text] + [
        re.sub(r"[^a-zA-Z ]", "", b["title"]).lower()
        for b in books
    ]

    vectorizer = TfidfVectorizer(stop_words=STOP_WORDS)
    tfidf_matrix = vectorizer.fit_transform(documents)

    news_vector = tfidf_matrix[0]

    results = []

    for i, book in enumerate(books):
        book_vector = tfidf_matrix[i + 1]
        similarity = cosine_similarity(news_vector, book_vector)[0][0]

        matched = similarity >= threshold

        discounted_price = (
            round(book["price"] * 0.50, 2) if matched else book["price"]
        )

        results.append({
            "Title": book["title"],
            "Rating": book["rating"],
            "Original Price": book["price"],
            "Discounted Price": discounted_price,
            "Cosine Similarity": round(similarity, 3),
            "Matched": matched
        })

    return results


# =====================================================
# STEP 5: SAVE CSV
# =====================================================
def save_to_csv(books):
    pd.DataFrame(books).to_csv(
        "book_discount_based_on_news_cosine.csv",
        index=False
    )
    print("CSV saved as: book_discount_based_on_news_cosine.csv")


# =====================================================
# MAIN
# =====================================================
if __name__ == "__main__":
    print("Scraping books...")
    books = scrape_all_books()

    print("Scraping news...")
    news = scrape_bbc_news()

    print("Applying cosine similarity discount...")
    discounted_books = apply_discount_using_cosine(books, news)

    save_to_csv(discounted_books)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Scraping books...
Scraping news...
Applying cosine similarity discount...
CSV saved as: book_discount_based_on_news_cosine.csv


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import re
from collections import Counter

# --- CONFIGURATION ---
BASE_URL = "https://books.toscrape.com/catalogue/"
PAGE_URL = "https://books.toscrape.com/catalogue/page-{}.html"
BBC_NEWS_URL = "https://www.bbc.com/news"
OUTPUT_FILE = "books_with_news_discounts.csv"
TOTAL_PAGES = 50

def get_bbc_keywords():
    """Fetches top 30 news URLs from BBC and extracts top keywords."""
    print("Fetching keywords from BBC News...")
    try:
        response = requests.get(BBC_NEWS_URL, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find article links
        links = []
        for a in soup.find_all('a', href=True):
            href = a['href']
            if "/news/" in href and not href.endswith("/news"):
                full_url = "https://www.bbc.com" + href if href.startswith('/') else href
                if full_url not in links:
                    links.append(full_url)
            if len(links) >= 30: break

        all_words = []
        stopwords = {'the', 'that', 'with', 'from', 'this', 'they', 'have', 'were', 'their', 'been', 'said', 'says', 'about'}

        for link in links:
            try:
                res = requests.get(link, timeout=5)
                article_soup = BeautifulSoup(res.text, 'html.parser')
                text = article_soup.get_text().lower()
                words = re.findall(r'\b[a-z]{4,}\b', text)
                all_words.extend([w for w in words if w not in stopwords])
            except: continue

        # Return top 50 most common keywords across all 30 articles
        return [word for word, count in Counter(all_words).most_common(50)]
    except Exception as e:
        print(f"Error fetching news: {e}")
        return []

def get_rating_value(rating_class):
    """Converts star rating class (e.g., 'Three') to a number."""
    ratings = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
    return ratings.get(rating_class, -1)

def scrape_books_with_logic():
    keywords = get_bbc_keywords()
    print(f"Top News Keywords: {keywords[:10]}...")

    with open(OUTPUT_FILE, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["Title", "Original Price", "Final Price", "Discount", "Rating", "Stock", "Author", "Review Snippet"])

        for page_num in range(1, TOTAL_PAGES + 1):
            print(f"Processing Page {page_num}...")
            res = requests.get(PAGE_URL.format(page_num))
            soup = BeautifulSoup(res.text, "html.parser")

            for pod in soup.find_all("article", class_="product_pod"):
                title = pod.h3.a["title"]

                # Navigate to detail page for Stock and Description
                detail_url = BASE_URL + pod.h3.a["href"].replace("../../../", "")
                detail_res = requests.get(detail_url)
                d_soup = BeautifulSoup(detail_res.text, "html.parser")

                # Get Description & Price (Remove £ symbol)
                desc = d_soup.select_one('article.product_page > p').text if d_soup.select_one('article.product_page > p') else ""
                price_raw = pod.find("p", class_="price_color").text
                price = float(re.sub(r'[^\d.]', '', price_raw))

                # Get Stock (Number)
                stock_text = d_soup.find("p", class_="instock availability").text.strip()
                stock_count = "".join(filter(str.isdigit, stock_text)) or "0"

                # Get Rating
                rating_class = pod.p['class'][1]
                rating_val = get_rating_value(rating_class)

                # DISCOUNT LOGIC
                match = any(word in desc.lower() for word in keywords)
                final_price = round(price * 0.5, 2) if match else price
                discount_applied = "50%" if match else "0%"

                # Write to CSV
                writer.writerow([title, price, final_price, discount_applied, rating_val, stock_count, "N/A", desc[:150]])

    print(f"Success! Data saved to {OUTPUT_FILE}")

if __name__ == "__main__":
    scrape_books_with_logic()

Fetching keywords from BBC News...
Top News Keywords: ['days', 'after', 'ukraine', 'gaza', 'will', 'russian', 'over', 'year', 'latest', 'trump']...
Processing Page 1...
Processing Page 2...
Processing Page 3...
Processing Page 4...
Processing Page 5...
Processing Page 6...
Processing Page 7...
Processing Page 8...
Processing Page 9...
Processing Page 10...
Processing Page 11...
Processing Page 12...
Processing Page 13...
Processing Page 14...
Processing Page 15...
Processing Page 16...
Processing Page 17...
Processing Page 18...
Processing Page 19...
Processing Page 20...
Processing Page 21...
Processing Page 22...
Processing Page 23...
Processing Page 24...
Processing Page 25...
Processing Page 26...
Processing Page 27...
Processing Page 28...
Processing Page 29...
Processing Page 30...
Processing Page 31...
Processing Page 32...
Processing Page 33...
Processing Page 34...
Processing Page 35...
Processing Page 36...
Processing Page 37...
Processing Page 38...
Processing Page 39...
Pro

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import re
from collections import Counter

# --- CONFIGURATION ---
BASE_URL = "https://books.toscrape.com/catalogue/"
PAGE_URL = "https://books.toscrape.com/catalogue/page-{}.html"
BBC_NEWS_URL = "https://www.bbc.com/news"
OUTPUT_FILE = "final_book_report.csv"
TOTAL_PAGES = 5  # Adjusted for testing; set to 50 for full site

def get_bbc_keywords():
    """Crawls 30 news articles from BBC to extract top 30 keywords."""
    print("Step 1: Extracting keywords from 30 BBC News articles...")
    try:
        response = requests.get(BBC_NEWS_URL, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')

        links = []
        for a in soup.find_all('a', href=True):
            href = a['href']
            if "/news/" in href and not href.endswith("/news"):
                full_url = "https://www.bbc.com" + href if href.startswith('/') else href
                if full_url not in links: links.append(full_url)
            if len(links) >= 30: break

        all_text = ""
        for link in links:
            try:
                res = requests.get(link, timeout=5)
                article_soup = BeautifulSoup(res.text, 'html.parser')
                all_text += " " + article_soup.get_text().lower()
            except: continue

        words = re.findall(r'\b[a-z]{5,}\b', all_text) # Words with 5+ letters
        stopwords = {'their', 'would', 'about', 'there', 'people', 'which', 'could'}
        filtered = [w for w in words if w not in stopwords]
        return [word for word, count in Counter(filtered).most_common(30)]
    except Exception as e:
        print(f"BBC Error: {e}")
        return []

def get_real_book_metadata(title):
    """Fetches real Author, Popularity (Rating), and Review snippet from Google Books API."""
    try:
        api_url = f"https://www.googleapis.com/books/v1/volumes?q=intitle:{title.replace(' ', '+')}"
        data = requests.get(api_url, timeout=5).json()
        if "items" in data:
            volume = data["items"][0]["volumeInfo"]
            return {
                "author": ", ".join(volume.get("authors", ["Unknown Author"])),
                "popularity": volume.get("averageRating", "N/A"),
                "review": volume.get("description", "No review found")[:150] + "..."
            }
    except: pass
    return {"author": "Unknown", "popularity": "N/A", "review": "N/A"}

def scrape_and_process():
    news_keywords = get_bbc_keywords()
    print(f"Top Keywords to match: {news_keywords}\n")

    with open(OUTPUT_FILE, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["Title", "Author", "Original Price", "Final Price", "Discount Status", "Popularity (0-5)", "Rating (Stars)", "Stock", "Review Snippet"])

        for page_num in range(1, TOTAL_PAGES + 1):
            print(f"Scraping Books Page {page_num}...")
            res = requests.get(PAGE_URL.format(page_num))
            soup = BeautifulSoup(res.text, "html.parser")

            for pod in soup.find_all("article", class_="product_pod"):
                title = pod.h3.a["title"]

                # 1. Get Real Metadata (Author/Popularity)
                real_info = get_real_book_metadata(title)

                # 2. Get Detail Page Info (Stock/Description)
                detail_url = BASE_URL + pod.h3.a["href"].replace("../../../", "")
                d_res = requests.get(detail_url)
                d_soup = BeautifulSoup(d_res.text, "html.parser")

                desc = d_soup.select_one('article.product_page > p').text.lower() if d_soup.select_one('article.product_page > p') else ""
                price_raw = pod.find("p", class_="price_color").text
                original_price = float(re.sub(r'[^\d.]', '', price_raw))

                stock_text = d_soup.find("p", class_="instock availability").get_text(strip=True)
                stock_count = "".join(filter(str.isdigit, stock_text)) or "0"

                star_map = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
                star_rating = star_map.get(pod.p['class'][1], -1)

                # 3. DISCOUNT LOGIC: Only apply if keyword matches description
                match = any(word in desc for word in news_keywords)
                if match:
                    final_price = round(original_price * 0.5, 2)
                    status = "50% OFF (Match)"
                else:
                    final_price = original_price
                    status = "No Match (Original Price)"

                writer.writerow([
                    title, real_info['author'], original_price, final_price,
                    status, real_info['popularity'], star_rating, stock_count, real_info['review']
                ])
                time.sleep(0.5) # Prevent API rate limits

    print(f"\nCompleted! Check '{OUTPUT_FILE}' for the results.")

if __name__ == "__main__":
    scrape_and_process()

Step 1: Extracting keywords from 30 BBC News articles...
Top Keywords to match: ['after', 'ukraine', 'russian', 'latest', 'trump', 'russia', 'external', 'broadcasting', 'warwar', 'picturesbbc', 'killed', 'americamiddle', 'ukraineus', 'eastin', 'indepthbbc', 'ireland', 'military', 'years', 'other', 'president', 'british', 'putin', 'since', 'minister', 'police', 'attack', 'bondi', 'christmas', 'rights', 'university']

Scraping Books Page 1...
Scraping Books Page 2...
Scraping Books Page 3...
Scraping Books Page 4...
Scraping Books Page 5...

Completed! Check 'final_book_report.csv' for the results.
